<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Manipulated_Media_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deepfake Detection and Multimodal Manipulated Media Analysis Pipeline for Defensive Forensic**



This end-to-end pipeline uses LangChain-based VLLM and Groq LLM integration
along with LangGraph multiagent orchestration to analyze audio, video, and image modalities.
Each modality has its own processing and analysis steps, and the results are aggregated
into a consolidated report

In [1]:
!pip install -q torch numpy opencv-python librosa pydantic mediapipe moviepy face_recognition scikit-image dtw-python scipy langchain langchain_community langchain_core langgraph nest_asyncio ipywidgets

In [2]:
#########################
###  IMPORTS SECTION  ###
#########################
import os
import torch
import cv2
import numpy as np
import librosa
import asyncio
import json
import re
import gc
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from typing import Dict, List, Any, Tuple, Optional

# Media processing libraries
import mediapipe as mp
from pydantic import BaseModel, Field
from moviepy.editor import VideoFileClip

# For interactive notebook usage
import nest_asyncio
import ipywidgets as widgets
from IPython.display import display, clear_output

# For image quality metrics
from skimage.metrics import structural_similarity as ssim
from dtw import dtw
import face_recognition
from scipy.spatial.distance import cosine

# Langchain and LangGraph imports
from langchain_community.llms import VLLM
from langchain_core.language_models.llms import LLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langgraph.graph import StateGraph, END

# Apply nest_asyncio to allow running async code in notebooks
nest_asyncio.apply()
torch.cuda.empty_cache()

  if event.key is 'enter':



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
#############################
### GLOBAL DEBUG SETTINGS ###
#############################
DEBUG = True
def debug_print(msg: str):
    """Print debug messages when DEBUG is enabled."""
    if DEBUG:
        print(f"[DEBUG] {msg}")

In [4]:
######################################
### MODEL INITIALIZATION SECTION  ###
######################################
COMMON_PARAMS = {
    "task": "generate",
    "max_model_len": 4096,
    "dtype": "half",
    "gpu_memory_utilization": 0.85,
    "cpu_offload_gb": 8,
    "enforce_eager": True,
    "trust_remote_code": True
}

def init_vllm_model(name: str, model_id: str, **overrides):
    """Initialize a VLLM model with specified parameters."""
    params = {**COMMON_PARAMS, **overrides}
    debug_print(f"Initializing VLLM model '{name}' with id '{model_id}' and params: {params}")
    return {"name": name, "model_id": model_id, "params": params}

def init_groq_model(name: str, model_id: str):
    """Initialize a Groq model with API key."""
    api_key = os.environ.get("GROQ_API_KEY", "your_groq_api_key")
    debug_print(f"Initializing Groq model '{name}' with id '{model_id}' using API key.")
    return {"name": name, "model_id": model_id, "api_key": api_key}

# Groq LLM wrapper class for LangChain
class GroqLLM:
    """A wrapper for Groq LLM API."""
    def __init__(self, model_data):
        self.model_data = model_data

    def call_as_llm(self, prompt: str) -> str:
        """Process a prompt through the Groq API."""
        debug_print(f"GroqLLM processing prompt with {self.model_data['name']}")
        # In a real implementation, this would make an API call to Groq
        # Here we simulate a response for demonstration
        if "audio" in self.model_data["name"]:
            return "Score: 0.82\nAnomalies: [\"unnatural voice transitions\", \"inconsistent background noise\"]"
        elif "video" in self.model_data["name"]:
            return "Score: 0.77\nAnomalies: [\"facial landmark inconsistencies\", \"unnatural eye movements\"]"
        elif "image" in self.model_data["name"]:
            return "Score: 0.68\nAnomalies: [\"lighting inconsistencies\", \"unusual facial proportions\"]"
        else:
            return "Score: 0.75\nAnomalies: [\"inconsistent narrative\", \"unusual phrasing patterns\"]"

class GroqLLMWrapper(LLM):
    """LangChain compatible wrapper for GroqLLM."""
    def __init__(self, groq_llm: GroqLLM, **kwargs):
        super().__init__(**kwargs)
        self._groq_llm = groq_llm

    @property
    def _llm_type(self) -> str:
        return "groq_llm"

    def _call(self, prompt: str, stop=None) -> str:
        return self._groq_llm.call_as_llm(prompt)

# Simulated VLLM model wrapper
class VLLMSimulated:
    """A simulated VLLM model for demonstration."""
    def __init__(self, model_data):
        self.model_data = model_data

    def __call__(self, prompt: str) -> str:
        """Process a prompt through the simulated VLLM."""
        debug_print(f"VLLM simulated processing with {self.model_data['name']}")
        # Simulate different responses based on the model name
        if "wav2vec" in self.model_data["name"] or "whisper" in self.model_data["name"]:
            return "Score: 0.79\nAnomalies: [\"frequency anomalies\", \"unnatural pauses\"]"
        elif "video" in self.model_data["name"]:
            return "Score: 0.81\nAnomalies: [\"temporal inconsistencies\", \"blending artifacts\"]"
        elif "llava" in self.model_data["name"] or "clip" in self.model_data["name"]:
            return "Score: 0.73\nAnomalies: [\"compression artifacts\", \"unusual texture patterns\"]"
        else:
            return "Score: 0.78\nAnomalies: [\"stylistic inconsistencies\"]"

# Initialize models for each modality
debug_print("Initializing modality-specific models...")
models = {
    "audio": {
        "vllm": [
            VLLMSimulated(init_vllm_model("wav2vec2", "facebook/wav2vec2-large-robust-ft-swbd-300h", tensor_parallel_size=1)),
            VLLMSimulated(init_vllm_model("whisper", "openai/whisper-large-v3", tensor_parallel_size=2))
        ],
        "groq": [
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_audio_model", "whisper-large-v3-turbo"))),
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_llama_audio", "meta-llama/llama-4-audio-17b-16e-instruct")))
        ]
    },
    "video": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llava_next_video", "llava-hf/LLaVA-NeXT-Video-7B-hf", tensor_parallel_size=2, max_tokens=1024)),
            VLLMSimulated(init_vllm_model("videomae", "MCG-NJU/videomae-large-static", tensor_parallel_size=2))
        ],
        "groq": [
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_video_scout", "meta-llama/llama-4-scout-17b-16e-instruct"))),
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_video_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct")))
        ]
    },
    "image": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llava_image", "llava-hf/llava-onevision-qwen2-7b-ov-hf", tensor_parallel_size=2)),
            VLLMSimulated(init_vllm_model("clip", "openai/clip-vit-large-patch14", tensor_parallel_size=1))
        ],
        "groq": [
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_image_scout", "meta-llama/llama-4-scout-17b-16e-instruct"))),
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_image_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct")))
        ]
    },
    "text": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llama3", "meta-llama/Llama-3-70b-hf", tensor_parallel_size=4))
        ],
        "groq": [
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_text_scout", "meta-llama/llama-4-scout-17b-16e-instruct"))),
            GroqLLMWrapper(GroqLLM(init_groq_model("groq_text_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct")))
        ]
    }
}

[DEBUG] Initializing modality-specific models...
[DEBUG] Initializing VLLM model 'wav2vec2' with id 'facebook/wav2vec2-large-robust-ft-swbd-300h' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 1}
[DEBUG] Initializing VLLM model 'whisper' with id 'openai/whisper-large-v3' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 2}
[DEBUG] Initializing Groq model 'groq_audio_model' with id 'whisper-large-v3-turbo' using API key.
[DEBUG] Initializing Groq model 'groq_llama_audio' with id 'meta-llama/llama-4-audio-17b-16e-instruct' using API key.
[DEBUG] Initializing VLLM model 'llava_next_video' with id 'llava-hf/LLaVA-NeXT-Video-7B-hf' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype

In [5]:
###############################################
### DATA MODELS & REPORT STRUCTURES SECTION ###
###############################################
class DeepfakeAnalysisResult(BaseModel):
    """Model for storing deepfake analysis results per modality."""
    score: float
    label: str
    anomalies: List[str] = Field(default_factory=list)
    artifacts: List[str] = Field(default_factory=list)
    confidence: float
    method: str
    timestamp: datetime = Field(default_factory=datetime.now)
    explanation: Optional[str] = None
    model_scores: Dict[str, float] = Field(default_factory=dict)

class Evidence(BaseModel):
    """Model for storing specific evidence of manipulation."""
    type: str
    description: str
    confidence: float
    method: str
    timestamp: Optional[float] = None
    location: Optional[Dict[str, int]] = None

class MultimodalAnalysisReport(BaseModel):
    """Comprehensive report aggregating all modality analyses."""
    case_id: str
    file_info: Dict[str, Any]
    video_analysis: Optional[DeepfakeAnalysisResult]
    audio_analysis: Optional[DeepfakeAnalysisResult]
    image_analysis: Optional[DeepfakeAnalysisResult]
    text_analysis: Optional[DeepfakeAnalysisResult]
    multimodal_score: float
    verdict: str
    evidence: List[Evidence]
    metadata: Dict[str, Any]
    recommendations: List[str] = Field(default_factory=list)
    confidence_matrix: Dict[str, Dict[str, float]] = Field(default_factory=dict)
    processing_time: float

In [6]:
#######################################
### PROCESSING & ANALYSIS FUNCTIONS ###
#######################################
# --- Shared utility functions ---
def parse_model_output(output: str) -> Tuple[float, List[str]]:
    """Parse model output to extract score and anomalies."""
    debug_print(f"Parsing model output: {output}")
    score_match = re.search(r"Score:?\s*(0\.\d+|1\.0)", output, re.IGNORECASE)
    score = float(score_match.group(1)) if score_match else 0.5
    anomalies = []
    anomalies_pattern = r"Anomalies:?\s*\[(.*?)\]|Anomalies:?\s*(.*?)(?=\n|$)"
    anomalies_match = re.search(anomalies_pattern, output, re.IGNORECASE | re.DOTALL)
    if anomalies_match:
        anomalies_text = anomalies_match.group(1) or anomalies_match.group(2)
        if anomalies_text:
            anomalies = [a.strip().strip('"\'') for a in anomalies_text.split(',') if a.strip()]
    debug_print(f"Parsed output: score={score}, anomalies={anomalies}")
    return score, anomalies

async def aggregate_llm_outputs(prompt: str, content: str, modality: str="generic") -> Tuple[float, List[str], Dict[str, float]]:
    """Aggregate outputs from multiple LLMs for a given modality."""
    debug_print(f"Aggregating LLM outputs for modality {modality}")
    scores = []
    all_anomalies = []
    model_scores = {}

    try:
        # Get responses from VLLM models
        for i, model in enumerate(models[modality]["vllm"]):
            model_name = f"vllm_{modality}_{i}"
            try:
                response = model(f"{prompt}\n\nContent: {content}")
                s, a = parse_model_output(response)
                scores.append(s)
                all_anomalies.extend(a)
                model_scores[model_name] = s
                debug_print(f"{model_name} score: {s}, anomalies: {a}")
            except Exception as e:
                debug_print(f"Error with {model_name}: {e}")

        # Get responses from Groq models
        for i, model in enumerate(models[modality]["groq"]):
            model_name = f"groq_{modality}_{i}"
            try:
                chat_prompt = ChatPromptTemplate.from_messages([
                    ("system", f"You are a forensic {modality} deepfake detection expert."),
                    ("user", f"{prompt}\n\nAnalyze this content: {content}")
                ])
                chain = LLMChain(prompt=chat_prompt, llm=model)
                response = await asyncio.to_thread(chain.run, {"content": content})
                s, a = parse_model_output(response)
                scores.append(s)
                all_anomalies.extend(a)
                model_scores[model_name] = s
                debug_print(f"{model_name} score: {s}, anomalies: {a}")
            except Exception as e:
                debug_print(f"Error with {model_name}: {e}")

    except Exception as e:
        debug_print(f"Error in aggregate_llm_outputs: {e}")

    # Compute aggregate score and deduplicate anomalies
    agg_score = float(np.mean(scores)) if scores else 0.5
    unique_anomalies = list(set(all_anomalies))
    debug_print(f"Aggregated score for {modality}: {agg_score}, anomalies: {unique_anomalies}")
    return agg_score, unique_anomalies, model_scores

In [7]:
# --- AUDIO PROCESSING & ANALYSIS ---
def audio_preprocessing(audio_path: str) -> Tuple[np.ndarray, int]:
    """Load and preprocess audio data."""
    debug_print(f"Loading and preprocessing audio from {audio_path} ...")
    try:
        audio_data, sr = librosa.load(audio_path, sr=16000)

        # Basic preprocessing steps
        # 1. Normalization
        audio_data = audio_data / np.max(np.abs(audio_data))

        # 2. Simple noise reduction (high-pass filter)
        if len(audio_data) > 0:
            b, a = librosa.filters.butter(4, 100/(sr/2), btype='highpass')
            audio_data = librosa.filters.filtfilt(b, a, audio_data)

        debug_print(f"Audio loaded and preprocessed. Sample rate: {sr}, Duration: {len(audio_data)/sr:.2f}s")
        return audio_data, sr
    except Exception as e:
        debug_print(f"Error in audio preprocessing: {e}")
        raise

def extract_audio_features(audio_data: np.ndarray, sr: int) -> Dict[str, Any]:
    """Extract relevant features from audio data for deepfake detection."""
    debug_print("Extracting audio features...")
    features = {}

    # Short-time Fourier transform
    stft = np.abs(librosa.stft(audio_data))
    features["stft"] = stft

    # Mel spectrogram
    mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    features["mel_spectrogram"] = mel_spec

    # MFCC features
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
    features["mfccs"] = mfccs

    # Spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sr)
    features["spectral_contrast"] = spectral_contrast

    # Chroma features
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
    features["chroma"] = chroma

    # Zero crossing rate
    zcr = librosa.feature.zero_crossing_rate(audio_data)
    features["zero_crossing_rate"] = zcr

    # Tempo estimation
    onset_env = librosa.onset.onset_strength(y=audio_data, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    features["tempo"] = tempo[0]

    debug_print(f"Audio features extracted: {list(features.keys())}")
    return features

def detect_audio_anomalies(features: Dict[str, Any]) -> List[str]:
    """Detect anomalies in audio features that might indicate deepfakes."""
    debug_print("Detecting audio anomalies from features...")
    anomalies = []

    # Example anomaly detection logic (would be much more sophisticated in practice)

    # Check for unusual spectral patterns
    mel_spec = features["mel_spectrogram"]
    if np.std(mel_spec) < 0.1 or np.std(mel_spec) > 10:
        anomalies.append("unusual spectral distribution")

    # Check for unnatural MFCC patterns
    mfccs = features["mfccs"]
    if np.max(np.diff(mfccs, axis=1)) > 5:
        anomalies.append("abrupt MFCC transitions")

    # Check for unusual tempo
    tempo = features["tempo"]
    if tempo < 40 or tempo > 240:
        anomalies.append("unusual speech tempo")

    # Check for unusual zero-crossing rate
    zcr = features["zero_crossing_rate"]
    if np.mean(zcr) > 0.3:
        anomalies.append("unusually high zero-crossing rate")

    debug_print(f"Detected audio anomalies: {anomalies}")
    return anomalies

async def advanced_audio_analysis(audio_data: np.ndarray, sr: int, device: torch.device) -> DeepfakeAnalysisResult:
    """Perform comprehensive audio analysis for deepfake detection."""
    debug_print("Starting advanced audio analysis...")

    # Extract audio features
    audio_features = extract_audio_features(audio_data, sr)

    # Detect anomalies from traditional signal processing
    signal_anomalies = detect_audio_anomalies(audio_features)
    signal_score = 0.8 if not signal_anomalies else 0.6  # Simplified scoring

    # Format feature content for LLM analysis
    feature_summary = (
        f"Audio duration: {len(audio_data)/sr:.2f}s, Sample rate: {sr}Hz\n"
        f"Mean amplitude: {np.mean(np.abs(audio_data)):.4f}, Max amplitude: {np.max(np.abs(audio_data)):.4f}\n"
        f"Detected tempo: {audio_features['tempo']:.2f} BPM\n"
        f"Mean zero-crossing rate: {np.mean(audio_features['zero_crossing_rate']):.4f}\n"
        f"Signal-to-noise ratio estimate: {np.mean(audio_data**2) / np.std(audio_data):.4f}\n"
        f"Traditional analysis anomalies: {signal_anomalies}"
    )

    # Get LLM-based analysis
    prompt = "Based on the audio features, provide a deepfake confidence score (0-1) and list specific audio anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="audio")

    # Combine traditional and LLM analysis
    combined_anomalies = list(set(signal_anomalies + llm_anomalies))
    final_score = (signal_score + llm_score) / 2
    confidence = 1.0 - np.std([signal_score, llm_score])  # Higher agreement = higher confidence

    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_audio_analysis (signal processing + LLM)",
        anomalies=combined_anomalies,
        explanation="Advanced audio analysis combining signal processing features and multiagent LLM insights.",
        model_scores=model_scores
    )

In [8]:
# --- VIDEO PROCESSING & ANALYSIS ---
def video_preprocessing(video_path: str) -> Dict[str, Any]:
    """Load and preprocess video data."""
    debug_print(f"Loading video from {video_path} ...")
    try:
        video = VideoFileClip(video_path)

        # Extract frames at regular intervals
        frame_count = int(video.fps * video.duration)
        sample_rate = max(1, frame_count // 30)  # Extract up to 30 frames
        frames = [video.get_frame(i/video.fps) for i in range(0, frame_count, sample_rate)]

        # Extract audio if available
        audio = video.audio.to_soundarray() if video.audio else None

        # Collect metadata
        metadata = {
            "duration": video.duration,
            "fps": video.fps,
            "frame_count": frame_count,
            "resolution": f"{video.size[0]}x{video.size[1]}",
            "file_size": os.path.getsize(video_path),
            "file_path": video_path
        }

        debug_print(f"Video preprocessed: {len(frames)} frames extracted, Audio present: {'Yes' if audio is not None else 'No'}")
        return {
            "frames": frames,
            "audio": audio,
            "metadata": metadata,
            "image": frames[0] if frames else None,
            "text": "Extracted text placeholder"
        }
    except Exception as e:
        debug_print(f"Error in video preprocessing: {e}")
        raise

def extract_video_features(frames: List[np.ndarray]) -> Dict[str, Any]:
    """Extract relevant features from video frames for deepfake detection."""
    debug_print(f"Extracting video features from {len(frames)} frames...")
    features = {}

    # Initialize face mesh detector for facial landmarks
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        min_detection_confidence=0.5
    )

    # Process frames
    face_landmarks_sequence = []
    face_embeddings = []
    optical_flow_metrics = []
    frame_diffs = []
    blur_metrics = []
    compression_metrics = []

    for i in range(len(frames)):
        # Convert to RGB for face detection
        frame_rgb = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB) if frames[i].shape[2] == 3 else frames[i]

        # Get facial landmarks
        face_landmarks = face_mesh.process(frame_rgb).multi_face_landmarks
        if face_landmarks:
            face_landmarks_sequence.append(face_landmarks[0].landmark)

            # Get face location for face recognition
            face_locations = face_recognition.face_locations(frame_rgb)
            if face_locations:
                # Get face encoding
                face_encoding = face_recognition.face_encodings(frame_rgb, face_locations)[0]
                face_embeddings.append(face_encoding)

        # Calculate frame differences (if not the first frame)
        if i > 0:
            # Convert frames to grayscale for simpler comparison
            prev_gray = cv2.cvtColor(frames[i-1], cv2.COLOR_BGR2GRAY)
            curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)

            # Frame difference
            diff = cv2.absdiff(prev_gray, curr_gray)
            frame_diffs.append(np.mean(diff))

            # Simple optical flow metric
            flow = np.mean(diff)  # In a real implementation, use proper optical flow
            optical_flow_metrics.append(flow)

        # Calculate blur metric (variance of Laplacian)
        gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        blur_metric = cv2.Laplacian(gray, cv2.CV_64F).var()
        blur_metrics.append(blur_metric)

        # Simple compression artifacts metric
        compression_metric = np.mean(cv2.Canny(gray, 100, 200))
        compression_metrics.append(compression_metric)

    # Store features
    features["face_landmarks_sequence"] = face_landmarks_sequence
    features["face_embeddings"] = face_embeddings
    features["optical_flow_metrics"] = optical_flow_metrics
    features["frame_diffs"] = frame_diffs
    features["blur_metrics"] = blur_metrics
    features["compression_metrics"] = compression_metrics

    debug_print(f"Video features extracted: {list(features.keys())}")
    debug_print(f"Found facial landmarks in {len(face_landmarks_sequence)}/{len(frames)} frames")

    face_mesh.close()
    return features

def detect_video_anomalies(features: Dict[str, Any]) -> List[str]:
    """Detect anomalies in video features that might indicate deepfakes."""
    debug_print("Detecting video anomalies from features...")
    anomalies = []

    # Check for facial landmark consistency
    face_landmarks = features.get("face_landmarks_sequence", [])
    if len(face_landmarks) > 1:
        # Check for abrupt changes in landmarks
        landmark_diffs = []
        for i in range(1, len(face_landmarks)):
            diff = sum(np.sqrt((face_landmarks[i][j].x - face_landmarks[i-1][j].x)**2 +
                      (face_landmarks[i][j].y - face_landmarks[i-1][j].y)**2)
                     for j in range(min(5, len(face_landmarks[i]))))  # Use first 5 landmarks
            landmark_diffs.append(diff)

        if np.max(landmark_diffs) > 0.5:  # Threshold for unnatural movement
            anomalies.append("abrupt facial landmark movements")

    # Check face embedding consistency
    face_embeddings = features.get("face_embeddings", [])
    if len(face_embeddings) > 1:
        embedding_diffs = []
        for i in range(1, len(face_embeddings)):
            diff = cosine(face_embeddings[i], face_embeddings[i-1])
            embedding_diffs.append(diff)

        if np.max(embedding_diffs) > 0.3:  # Threshold for identity shift
            anomalies.append("inconsistent face identity")

    # Check for unnatural motion
    frame_diffs = features.get("frame_diffs", [])
    if frame_diffs:
        if np.std(frame_diffs) < 0.01:  # Too smooth
            anomalies.append("unnaturally smooth motion")
        if np.max(frame_diffs) / (np.mean(frame_diffs) + 1e-6) > 10:  # Erratic changes
            anomalies.append("erratic frame changes")

    # Check for inconsistent blur levels
    blur_metrics = features.get("blur_metrics", [])
    if blur_metrics:
        if np.std(blur_metrics) / (np.mean(blur_metrics) + 1e-6) > 0.8:  # Inconsistent blur
            anomalies.append("inconsistent blur levels")

    # Check for compression artifact patterns
    compression_metrics = features.get("compression_metrics", [])
    if compression_metrics:
        if np.std(compression_metrics) / (np.mean(compression_metrics) + 1e-6) > 0.5:
            anomalies.append("inconsistent compression artifacts")

    debug_print(f"Detected video anomalies: {anomalies}")
    return anomalies

async def advanced_video_analysis(video_data: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    """Perform comprehensive video analysis for deepfake detection."""
    debug_print("Starting advanced video analysis...")

    frames = video_data.get("frames", [])
    if not frames:
        debug_print("No frames available for analysis")
        return DeepfakeAnalysisResult(
            score=0.5,
            label="INCONCLUSIVE",
            confidence=0.0,
            method="advanced_video_analysis",
            anomalies=["no frames available for analysis"],
            explanation="Could not perform video analysis due to lack of frame data."
        )

    # Extract video features
    video_features = extract_video_features(frames)

    # Detect anomalies from traditional computer vision
    cv_anomalies = detect_video_anomalies(video_features)
    cv_score = 0.8 if not cv_anomalies else 0.6  # Simplified scoring

    # Format feature content for LLM analysis
    feature_summary = (
        f"Video features: {len(frames)} frames, Resolution: {video_data['metadata']['resolution']}\n"
        f"Facial landmarks detected in {len(video_features.get('face_landmarks_sequence', []))}/{len(frames)} frames\n"
        f"Mean frame difference: {np.mean(video_features.get('frame_diffs', [0])):.4f}\n"
        f"Blur consistency: {np.std(video_features.get('blur_metrics', [0]))/(np.mean(video_features.get('blur_metrics', [1]))+1e-6):.4f}\n"
        f"Traditional analysis anomalies: {cv_anomalies}"
    )

    # Get LLM-based analysis
    prompt = "Based on the video features, provide a deepfake confidence score (0-1) and list specific video anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="video")

    # Combine traditional and LLM analysis
    combined_anomalies = list(set(cv_anomalies + llm_anomalies))
    final_score = (cv_score + llm_score) / 2
    confidence = 1.0 - np.std([cv_score, llm_score])  # Higher agreement = higher confidence

    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_video_analysis (computer vision + LLM)",
        anomalies=combined_anomalies,
        explanation="Advanced video analysis combining computer vision features and multiagent LLM insights.",
        model_scores=model_scores
    )

In [9]:
# --- IMAGE PROCESSING & ANALYSIS ---
def image_preprocessing(image_path: str) -> np.ndarray:
    """Load and preprocess image data."""
    debug_print(f"Loading and preprocessing image from {image_path} ...")
    try:
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image from {image_path}")

        # Resize if too large
        max_dim = 1024
        h, w = image.shape[:2]
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            image = cv2.resize(image, (int(w * scale), int(h * scale)))
            debug_print(f"Image resized from {w}x{h} to {int(w * scale)}x{int(h * scale)}")

        # Convert to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        debug_print(f"Image loaded and preprocessed. Shape: {image_rgb.shape}")
        return image_rgb
    except Exception as e:
        debug_print(f"Error in image preprocessing: {e}")
        raise

def extract_image_features(image: np.ndarray) -> Dict[str, Any]:
    """Extract relevant features from an image for deepfake detection."""
    debug_print("Extracting image features...")
    features = {}

    # Initialize face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    features["face_count"] = len(faces)

    # If a face is detected, extract face-specific features
    if len(faces) > 0:
        # Extract the largest face
        x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])
        face_roi = gray[y:y+h, x:x+w]
        features["face_roi"] = face_roi

        # Face landmarks using face_recognition
        face_landmarks = face_recognition.face_landmarks(image)
        features["face_landmarks"] = face_landmarks

        # Face encoding (for identity verification)
        face_encodings = face_recognition.face_encodings(image)
        if face_encodings:
            features["face_encoding"] = face_encodings[0]

    # General image features
    # Error Level Analysis (ELA) - simplified version
    # Save image with low quality and reload
    temp_path = "temp_ela.jpg"
    cv2.imwrite(temp_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 90])
    low_qual = cv2.imread(temp_path)
    low_qual_rgb = cv2.cvtColor(low_qual, cv2.COLOR_BGR2RGB)

    # Calculate difference
    ela = cv2.absdiff(image, low_qual_rgb)
    features["ela"] = cv2.cvtColor(ela, cv2.COLOR_RGB2GRAY)

    # Remove temp file
    if os.path.exists(temp_path):
        os.remove(temp_path)

    # Noise analysis
    noise = cv2.medianBlur(gray, 5) - gray
    features["noise_pattern"] = noise

    # Edge detection
    edges = cv2.Canny(gray, 100, 200)
    features["edges"] = edges

    # JPEG quantization tables - proxy using block artifacts
    dct_blocks = np.zeros_like(gray, dtype=float)
    block_size = 8
    for i in range(0, gray.shape[0] - block_size, block_size):
        for j in range(0, gray.shape[1] - block_size, block_size):
            block = gray[i:i+block_size, j:j+block_size].astype(float)
            dct_block = cv2.dct(block)
            dct_blocks[i:i+block_size, j:j+block_size] = dct_block

    features["dct_blocks"] = dct_blocks

    debug_print(f"Image features extracted. Faces found: {features['face_count']}")
    return features

def detect_image_anomalies(features: Dict[str, Any]) -> List[str]:
    """Detect anomalies in image features that might indicate deepfakes."""
    debug_print("Detecting image anomalies from features...")
    anomalies = []

    # Check for face-specific anomalies
    if features["face_count"] > 0:
        # Check face landmarks if available
        face_landmarks = features.get("face_landmarks", [])
        if face_landmarks:
            # Analyze symmetry (simplified)
            landmarks = face_landmarks[0]  # First face
            left_eye = np.mean(landmarks.get("left_eye", [[0, 0]]), axis=0)
            right_eye = np.mean(landmarks.get("right_eye", [[0, 0]]), axis=0)

            # Check horizontal alignment
            if abs(left_eye[1] - right_eye[1]) > 10:  # Eyes not horizontally aligned
                anomalies.append("asymmetric eye alignment")

            # Check proportions (simplified)
            nose_tip = landmarks.get("nose_tip", [[0, 0]])[0]
            top_lip = np.mean(landmarks.get("top_lip", [[0, 0]]), axis=0)
            if abs((nose_tip[1] - left_eye[1]) - (top_lip[1] - nose_tip[1])) > 15:
                anomalies.append("unusual facial proportions")

        # ELA analysis
        ela = features.get("ela")
        if ela is not None:
            ela_mean = np.mean(ela)
            ela_std = np.std(ela)

            # High ELA values can indicate manipulation
            if ela_mean > 10 or ela_std > 15:
                anomalies.append("potential image manipulation (ELA)")

        # Noise pattern analysis
        noise = features.get("noise_pattern")
        if noise is not None:
            noise_mean = np.mean(np.abs(noise))
            noise_std = np.std(noise)

            # Unnatural noise patterns
            if noise_std / (noise_mean + 1e-6) > 3 or noise_mean < 0.5:
                anomalies.append("unnatural noise pattern")

        # DCT block artifact analysis
        dct_blocks = features.get("dct_blocks")
        if dct_blocks is not None:
            # Look for unusual block patterns that could indicate splicing
            block_diff = cv2.Laplacian(dct_blocks, cv2.CV_64F).var()
            if block_diff > 500:
                anomalies.append("unusual compression artifacts")
    else:
        anomalies.append("no faces detected for analysis")

    debug_print(f"Detected image anomalies: {anomalies}")
    return anomalies

async def advanced_image_analysis(image: np.ndarray, device: torch.device) -> DeepfakeAnalysisResult:
    """Perform comprehensive image analysis for deepfake detection."""
    debug_print("Starting advanced image analysis...")

    # Extract image features
    image_features = extract_image_features(image)

    # Detect anomalies from traditional computer vision
    cv_anomalies = detect_image_anomalies(image_features)
    cv_score = 0.8 if not cv_anomalies else 0.6  # Simplified scoring

    # Format feature content for LLM analysis
    feature_summary = (
        f"Image features: Resolution: {image.shape[1]}x{image.shape[0]}, Faces detected: {image_features['face_count']}\n"
        f"ELA mean: {np.mean(image_features.get('ela', np.zeros(1))):.4f}, std: {np.std(image_features.get('ela', np.zeros(1))):.4f}\n"
        f"Noise pattern: mean: {np.mean(np.abs(image_features.get('noise_pattern', np.zeros(1)))):.4f}, std: {np.std(image_features.get('noise_pattern', np.zeros(1))):.4f}\n"
        f"Traditional analysis anomalies: {cv_anomalies}"
    )

    # Get LLM-based analysis
    prompt = "Based on the image features, provide a deepfake confidence score (0-1) and list specific image anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="image")

    # Combine traditional and LLM analysis
    combined_anomalies = list(set(cv_anomalies + llm_anomalies))
    final_score = (cv_score + llm_score) / 2
    confidence = 1.0 - np.std([cv_score, llm_score])  # Higher agreement = higher confidence

    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_image_analysis (computer vision + LLM)",
        anomalies=combined_anomalies,
        explanation="Advanced image analysis combining computer vision features and multiagent LLM insights.",
        model_scores=model_scores
    )

In [10]:
# --- TEXT ANALYSIS ---
async def analyze_text_content(text: str) -> DeepfakeAnalysisResult:
    """Analyze text for indicators of AI generation or manipulation."""
    debug_print("Starting text content analysis...")

    # Basic text analysis metrics
    word_count = len(text.split())
    sent_count = len(re.split(r'[.!?]+', text))
    avg_word_len = sum(len(word) for word in text.split()) / max(1, word_count)

    # Feature extraction for text
    feature_summary = (
        f"Text statistics: {word_count} words, {sent_count} sentences\n"
        f"Average word length: {avg_word_len:.2f}\n"
        f"Sample text (truncated): {text[:500]}...\n"
    )

    # Get LLM-based analysis
    prompt = "Based on the text features, provide a score (0-1) indicating if this text was AI-generated or manipulated and list specific text anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="text")

    return DeepfakeAnalysisResult(
        score=llm_score,
        label="HUMAN" if llm_score > 0.7 else "AI-GENERATED",
        confidence=0.8,  # Fixed confidence for demonstration
        method="text_analysis (LLM-based)",
        anomalies=llm_anomalies,
        explanation="Text analysis using multiagent LLM insights to detect AI-generated content.",
        model_scores=model_scores
    )

In [11]:
#######################################
###   MULTI-MODAL FUSION SECTION   ###
#######################################
def calculate_multimodal_score(results: Dict[str, DeepfakeAnalysisResult]) -> float:
    """Calculate an aggregate score from multiple modality results."""
    debug_print("Calculating multimodal score...")

    # Extract scores and weights based on confidence
    scores = []
    weights = []

    for modality, result in results.items():
        if result:
            scores.append(result.score)
            weights.append(result.confidence)

    # If no valid results, return default
    if not scores:
        debug_print("No valid results found for multimodal scoring")
        return 0.5

    # Normalize weights
    total_weight = sum(weights)
    if total_weight == 0:
        weighted_avg = sum(scores) / len(scores)
    else:
        norm_weights = [w / total_weight for w in weights]
        weighted_avg = sum(s * w for s, w in zip(scores, norm_weights))

    debug_print(f"Multimodal weighted score: {weighted_avg:.4f}")
    return weighted_avg

def generate_verdict(score: float, results: Dict[str, DeepfakeAnalysisResult]) -> str:
    """Generate a final verdict based on the multimodal score and individual results."""
    if score > 0.8:
        return "AUTHENTIC - High confidence that this content is genuine"
    elif score > 0.6:
        return "LIKELY AUTHENTIC - Moderate confidence that this content is genuine"
    elif score > 0.4:
        return "INCONCLUSIVE - Analysis unable to determine authenticity"
    elif score > 0.2:
        return "LIKELY MANIPULATED - Moderate confidence that this content contains manipulations"
    else:
        return "MANIPULATED - High confidence that this content is manipulated or synthetic"

def collect_evidence(results: Dict[str, DeepfakeAnalysisResult]) -> List[Evidence]:
    """Collect evidence from all modality results."""
    evidence_list = []

    for modality, result in results.items():
        if result:
            for anomaly in result.anomalies:
                evidence_list.append(
                    Evidence(
                        type=modality,
                        description=anomaly,
                        confidence=result.confidence,
                        method=result.method
                    )
                )

    return evidence_list

def generate_recommendations(report: MultimodalAnalysisReport) -> List[str]:
    """Generate recommendations based on the analysis report."""
    recommendations = []

    # Add general recommendation
    if report.multimodal_score < 0.4:
        recommendations.append("This content should be treated as potentially manipulated and not be used as evidence without further forensic analysis.")

    # Add modality-specific recommendations
    if report.video_analysis and report.video_analysis.score < 0.5:
        recommendations.append("Video content shows significant signs of manipulation and should be verified through alternative sources.")

    if report.audio_analysis and report.audio_analysis.score < 0.5:
        recommendations.append("Audio content appears to be synthetic or manipulated. Verify the source and content through other means.")

    if report.image_analysis and report.image_analysis.score < 0.5:
        recommendations.append("Image analysis indicates potential manipulation. Consider requesting the original, unprocessed image file.")

    if report.text_analysis and report.text_analysis.score < 0.5:
        recommendations.append("Text content appears to be AI-generated. Verify authorship through additional means.")

    # Add standard recommendations for all reports
    recommendations.append("For definitive analysis, consider submitting this content to a professional forensic lab specialized in digital media authentication.")

    return recommendations

In [12]:
#######################################
###   MAIN PIPELINE ORCHESTRATION  ###
#######################################
class DeepfakePipeline:
    """End-to-end pipeline for deepfake detection across multiple modalities."""

    def __init__(self):
        """Initialize the deepfake detection pipeline."""
        debug_print("Initializing DeepfakePipeline...")
        # Initialize device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        debug_print(f"Using device: {self.device}")

    async def analyze_file(self, file_path: str) -> MultimodalAnalysisReport:
        """Analyze a file for potential deepfake manipulation."""
        debug_print(f"Starting analysis for file: {file_path}")
        start_time = datetime.now()

        # Determine file type
        file_ext = os.path.splitext(file_path)[1].lower()
        file_info = {
            "path": file_path,
            "size": os.path.getsize(file_path),
            "type": "unknown",
            "extension": file_ext
        }

        # Initialize results
        results = {
            "video_analysis": None,
            "audio_analysis": None,
            "image_analysis": None,
            "text_analysis": None
        }

        try:
            # Process based on file type
            if file_ext in ['.mp4', '.mov', '.avi', '.mkv', '.webm']:
                file_info["type"] = "video"
                video_data = video_preprocessing(file_path)

                # Run modality analysis tasks concurrently
                tasks = []

                # Video analysis
                tasks.append(advanced_video_analysis(video_data, self.device))

                # Audio analysis if available
                if video_data.get("audio") is not None:
                    audio_data, sr = video_data["audio"], 44100  # Assuming standard sample rate
                    tasks.append(advanced_audio_analysis(audio_data, sr, self.device))

                # Extract representative image from first frame
                if video_data.get("frames"):
                    image = video_data["frames"][0]
                    tasks.append(advanced_image_analysis(image, self.device))

                # Extract and analyze text if available (e.g., from speech recognition)
                if video_data.get("text"):
                    tasks.append(analyze_text_content(video_data["text"]))

                # Wait for all analysis tasks to complete
                completed_tasks = await asyncio.gather(*tasks)

                # Assign results
                results["video_analysis"] = completed_tasks[0]
                task_idx = 1

                if video_data.get("audio") is not None:
                    results["audio_analysis"] = completed_tasks[task_idx]
                    task_idx += 1

                if video_data.get("frames"):
                    results["image_analysis"] = completed_tasks[task_idx]
                    task_idx += 1

                if video_data.get("text"):
                    results["text_analysis"] = completed_tasks[task_idx]

            elif file_ext in ['.wav', '.mp3', '.ogg', '.flac']:
                file_info["type"] = "audio"
                audio_data, sr = audio_preprocessing(file_path)
                results["audio_analysis"] = await advanced_audio_analysis(audio_data, sr, self.device)

                # Also analyze for potential text content (speech)
                text_content = "Audio transcript would be here in a real implementation."
                results["text_analysis"] = await analyze_text_content(text_content)

            elif file_ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                file_info["type"] = "image"
                image = image_preprocessing(file_path)
                results["image_analysis"] = await advanced_image_analysis(image, self.device)

            elif file_ext in ['.txt', '.md', '.rtf', '.doc', '.docx', '.pdf']:
                file_info["type"] = "text"
                # In a real implementation, extract text based on file type
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text_content = f.read()
                results["text_analysis"] = await analyze_text_content(text_content)

            else:
                debug_print(f"Unsupported file type: {file_ext}")
                raise ValueError(f"Unsupported file type: {file_ext}")

            # Calculate multimodal score
            multimodal_score = calculate_multimodal_score(results)

            # Generate verdict
            verdict = generate_verdict(multimodal_score, results)

            # Collect evidence
            evidence = collect_evidence(results)

            # Calculate processing time
            processing_time = (datetime.now() - start_time).total_seconds()

            # Create confidence matrix
            confidence_matrix = {
                modality: {
                    "score": result.score if result else None,
                    "confidence": result.confidence if result else None
                }
                for modality, result in results.items()
            }

            # Create report
            report = MultimodalAnalysisReport(
                case_id=f"case_{datetime.now().strftime('%Y%m%d%H%M%S')}",
                file_info=file_info,
                video_analysis=results["video_analysis"],
                audio_analysis=results["audio_analysis"],
                image_analysis=results["image_analysis"],
                text_analysis=results["text_analysis"],
                multimodal_score=multimodal_score,
                verdict=verdict,
                evidence=evidence,
                metadata={
                    "pipeline_version": "1.0.0",
                    "analysis_timestamp": datetime.now().isoformat(),
                    "device": str(self.device)
                },
                confidence_matrix=confidence_matrix,
                processing_time=processing_time
            )

            # Generate and add recommendations
            report.recommendations = generate_recommendations(report)

            return report

        except Exception as e:
            debug_print(f"Error in analyze_file: {e}")
            raise

In [13]:
#######################################
###  LANGGRAPH ORCHESTRATION GRAPH  ###
#######################################
class PipelineState(BaseModel):
    """State for the deepfake analysis pipeline graph."""
    file_path: str
    report: Optional[MultimodalAnalysisReport] = None
    current_step: str = "initialize"
    error: Optional[str] = None
    results: Dict[str, Any] = Field(default_factory=dict)

async def initialize_analysis(state: PipelineState) -> PipelineState:
    """Initialize the analysis process."""
    debug_print(f"Initializing analysis for {state.file_path}")
    try:
        state.current_step = "processing"
        return state
    except Exception as e:
        state.error = str(e)
        state.current_step = "error"
        return state

async def process_file(state: PipelineState) -> PipelineState:
    """Process the file using the DeepfakePipeline."""
    try:
        pipeline = DeepfakePipeline()
        state.report = await pipeline.analyze_file(state.file_path)
        state.current_step = "complete"
        return state
    except Exception as e:
        state.error = str(e)
        state.current_step = "error"
        return state

def create_pipeline_graph() -> StateGraph:
    """Create the LangGraph for the deepfake analysis pipeline."""
    workflow = StateGraph(PipelineState)

    # Add nodes
    workflow.add_node("initialize", initialize_analysis)
    workflow.add_node("processing", process_file)

    # Add edges
    workflow.add_edge("initialize", "processing")
    workflow.add_edge("processing", END)

    # Add conditional edge for error handling
    workflow.add_conditional_edges(
        "initialize",
        lambda state: "error" if state.error else "processing"
    )

    workflow.add_conditional_edges(
        "processing",
        lambda state: "error" if state.error else END
    )

    # Compile the graph
    return workflow.compile()

In [14]:
#######################################
###    MAIN APPLICATION SECTION     ###
#######################################
async def main():
    """Main entry point for the deepfake analysis pipeline."""
    import argparse

    parser = argparse.ArgumentParser(description="Deepfake Detection Pipeline")
    parser.add_argument('file_path', help='Path to the file to analyze')
    parser.add_argument('--debug', action='store_true', help='Enable debug output')
    parser.add_argument('--output', help='Path to save the JSON report')

    args = parser.parse_args()

    # Set debug mode
    global DEBUG
    DEBUG = args.debug

    debug_print(f"Starting deepfake analysis for file: {args.file_path}")

    try:
        # Create and run the pipeline graph
        workflow = create_pipeline_graph()
        state = PipelineState(file_path=args.file_path)
        final_state = await workflow.ainvoke(state)

        if final_state.error:
            print(f"Error: {final_state.error}")
            return 1

        # Display results
        report = final_state.report
        print("\n" + "="*50)
        print(f"DEEPFAKE ANALYSIS REPORT - {report.case_id}")
        print("="*50)
        print(f"File: {os.path.basename(args.file_path)} ({report.file_info['type']})")
        print(f"Overall Score: {report.multimodal_score:.2f} (0=Fake, 1=Real)")
        print(f"Verdict: {report.verdict}")
        print("-"*50)

        # Display modality results
        for modality in ["video", "audio", "image", "text"]:
            result = getattr(report, f"{modality}_analysis")
            if result:
                print(f"{modality.upper()} Analysis: {result.score:.2f} ({result.label})")
                if result.anomalies:
                    print(f"  Detected anomalies: {', '.join(result.anomalies)}")

        print("-"*50)
        print("RECOMMENDATIONS:")
        for i, rec in enumerate(report.recommendations, 1):
            print(f"{i}. {rec}")

        # Save JSON report if requested
        if args.output:
            with open(args.output, 'w') as f:
                f.write(report.json(indent=2))
            print(f"\nDetailed report saved to {args.output}")

        return 0

    except Exception as e:
        print(f"Error in main execution: {e}")
        import traceback
        traceback.print_exc()
        return 1

if __name__ == "__main__":
    asyncio.run(main())

usage: colab_kernel_launcher.py [-h] [--debug] [--output OUTPUT] file_path
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [ ]:
# Add this code at the end of the file, after the main() function

async def run_in_colab():
    """Run the deepfake detection pipeline in Google Colab with an interactive interface."""
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    import json
    import time

    # Header
    display(HTML("""
    <div style="background-color:#4285F4; padding:20px; border-radius:10px; margin-bottom:20px;">
        <h1 style="color:white; text-align:center;">Deepfake Detection Pipeline</h1>
        <p style="color:white; text-align:center;">Upload media files to analyze for potential manipulations</p>
    </div>
    """))

    # Upload widget
    upload_button = widgets.Button(
        description='Upload File',
        disabled=False,
        button_style='primary',
        tooltip='Click to upload a file for analysis',
        icon='upload'
    )

    debug_checkbox = widgets.Checkbox(
        value=True,
        description='Enable debug output',
        disabled=False
    )

    output_area = widgets.Output()
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=10,
        description='Processing:',
        bar_style='info',
        orientation='horizontal'
    )

    display(widgets.HBox([upload_button, debug_checkbox]))
    display(progress_bar)
    display(output_area)

    # Function to handle file upload and processing
    def on_upload_clicked(b):
        with output_area:
            clear_output()
            print("Please select a file to upload...")

            # Set global debug flag
            global DEBUG
            DEBUG = debug_checkbox.value

            try:
                uploaded = files.upload()

                if not uploaded:
                    print("No file was uploaded.")
                    return

                file_path = list(uploaded.keys())[0]
                print(f"Analyzing file: {file_path}")

                # Update progress bar
                progress_bar.value = 1

                # Create temporary path for the uploaded file
                temp_path = f"/tmp/{file_path}"
                with open(temp_path, "wb") as f:
                    f.write(uploaded[file_path])

                # Create and run the pipeline graph
                progress_bar.value = 3
                workflow = create_pipeline_graph()
                state = PipelineState(file_path=temp_path)

                # Run analysis
                print("Starting analysis... This may take a few minutes.")
                progress_bar.value = 5

                # Run the analysis asynchronously
                loop = asyncio.get_event_loop()
                final_state = loop.run_until_complete(workflow.ainvoke(state))

                progress_bar.value = 8

                if final_state.error:
                    print(f"Error: {final_state.error}")
                    return

                # Display results
                report = final_state.report
                print("\n" + "="*50)
                print(f"DEEPFAKE ANALYSIS REPORT")
                print("="*50)
                print(f"File: {file_path} ({report.file_info['type']})")
                print(f"Overall Score: {report.multimodal_score:.2f} (0=Fake, 1=Real)")
                print(f"Verdict: {report.verdict}")
                print("-"*50)

                # Display modality results
                for modality in ["video", "audio", "image", "text"]:
                    result = getattr(report, f"{modality}_analysis")
                    if result:
                        print(f"{modality.upper()} Analysis: {result.score:.2f} ({result.label})")
                        if result.anomalies:
                            print(f"  Detected anomalies: {', '.join(result.anomalies)}")

                print("-"*50)
                print("RECOMMENDATIONS:")
                for i, rec in enumerate(report.recommendations, 1):
                    print(f"{i}. {rec}")

                # Save JSON report to downloadable file
                report_json = report.json(indent=2)
                report_filename = f"deepfake_report_{int(time.time())}.json"
                with open(report_filename, 'w') as f:
                    f.write(report_json)

                print("\nDetailed JSON report available for download:")
                files.download(report_filename)

                # Create visualization of the results
                display(HTML(f"""
                <div style="background-color:#f5f5f5; padding:15px; border-radius:5px; margin-top:20px;">
                    <h3>Analysis Summary Visualization</h3>
                    <div style="display: flex; margin-bottom: 20px;">
                        <div style="flex: 1; text-align: center;">
                            <div style="font-size: 48px; font-weight: bold; color: {'green' if report.multimodal_score > 0.7 else 'red' if report.multimodal_score < 0.3 else 'orange'}">
                                {report.multimodal_score:.2f}
                            </div>
                            <div>Overall Score</div>
                        </div>
                        <div style="flex: 2; padding-left: 20px;">
                            <div style="font-size: 24px; margin-bottom: 10px;">{report.verdict}</div>
                            <div>{'✅ Likely authentic' if report.multimodal_score > 0.7 else '❌ Likely manipulated' if report.multimodal_score < 0.3 else '⚠️ Results inconclusive'}</div>
                        </div>
                    </div>

                    <h4>Modality Scores</h4>
                    <div style="display: flex; flex-wrap: wrap;">
                """))

                # Display modality scores as progress bars
                for modality in ["video", "audio", "image", "text"]:
                    result = getattr(report, f"{modality}_analysis")
                    if result:
                        score = result.score
                        color = 'green' if score > 0.7 else 'red' if score < 0.3 else 'orange'
                        display(HTML(f"""
                        <div style="flex: 1; min-width: 200px; margin: 10px;">
                            <div style="text-transform: uppercase; font-weight: bold;">{modality}</div>
                            <div style="background-color: #ddd; border-radius: 5px; height: 20px; margin: 5px 0;">
                                <div style="background-color: {color}; width: {int(score*100)}%; height: 100%; border-radius: 5px; text-align: right;">
                                    <span style="color: white; padding-right: 5px;">{score:.2f}</span>
                                </div>
                            </div>
                        </div>
                        """))

                display(HTML("</div></div>"))

                progress_bar.value = 10

            except Exception as e:
                import traceback
                print(f"Error in processing: {e}")
                traceback.print_exc()

    # Attach the handler to the button
    upload_button.on_click(on_upload_clicked)

    print("🚀 Ready! Click 'Upload File' to begin analysis.")

# Try to detect if we're running in Colab and run the appropriate interface
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if __name__ == "__main__":
    if IN_COLAB:
        import nest_asyncio
        nest_asyncio.apply()
        run_in_colab()
    else:
        asyncio.run(main())